In [16]:
from itertools import combinations

from instance import Instance
from sat import solve
from splitting_methods import first_choice
from splitting_methods import random_choice
from splitting_methods import two_clause_choice
import unit_preference_methods

In [17]:
def negate(literal):
    return (literal[1:] if literal.startswith('-') else '-' + literal)

In [18]:
# Test negate
lit_test = "horse_3"
print(negate(lit_test))

-horse_3


In [19]:
def combine(value, house):
    return f'{value}_{house}'

In [20]:
# Test combine
combine("horse", 4)

'horse_4'

In [21]:
def one_of(literals):
    return list(combinations(map(negate, literals), 2)) + list(combinations(literals, len(literals)))

In [22]:
# Test one of
lits_0 = ["horse_0", "horse_1", "horse_2", "horse_3", "horse_4"]
print(one_of(lits_0))

[('-horse_0', '-horse_1'), ('-horse_0', '-horse_2'), ('-horse_0', '-horse_3'), ('-horse_0', '-horse_4'), ('-horse_1', '-horse_2'), ('-horse_1', '-horse_3'), ('-horse_1', '-horse_4'), ('-horse_2', '-horse_3'), ('-horse_2', '-horse_4'), ('-horse_3', '-horse_4'), ('horse_0', 'horse_1', 'horse_2', 'horse_3', 'horse_4')]


In [23]:
def dnf_to_cnf(dnf):

    # We use frozensets to allow for sets to be hashed.
    # We want to use sets so we can create clauses without duplicate literals.
    cnf = {frozenset()}
    
    # Iterate over all dnf clauses.
    for and_clause in dnf:
        
        # Add new terms to each clause, check they're not unsat as we do it.
        cnf = {or_clause | set([literal]) for literal in and_clause for or_clause in cnf
              if negate(literal) not in or_clause}
        
        # Remove unnecessary terms
        shortest_clause = min(cnf, key=len)          
        cnf -= {or_clause for or_clause in cnf if or_clause > shortest_clause}
        
    # Return a list of tuples of clauses.
    return list(map(tuple, cnf))

In [24]:
# Test dnf to cnf conversion
dnf_test = [("a", "b", "c"), ("-b", "-c")]
print(dnf_to_cnf(dnf_test))

[('-c', 'b'), ('a', '-c'), ('a', '-b'), ('-b', 'c')]


In [25]:
def cnf_to_string(cnf):
    
    # Convert a list, tuple format to string format.
    cnf_str = ""
    
    for clause in cnf:
        clause_str = ""
        for lit in clause:
            clause_str += str(lit) + " "
        cnf_str += clause_str + "\n"
    return cnf_str

In [26]:
# List of house indices
houses = ['1', '2', '3', '4', '5']

# Functions to express puzzle dependencies
def found_at(value, house):
    return [(combine(value, house),)]

def same(a, b):
    return dnf_to_cnf([(combine(a, i), combine(b, i)) for i in houses])

def next_to(a, b):
    return dnf_to_cnf([(combine(a, i), combine(b, j)) for i, j in zip(houses, houses[1:])])

def either_side(a, b):
    return dnf_to_cnf([(combine(a, i), combine(b, j)) for i, j in zip(houses, houses[1:])] +
                    [(combine(b, i), combine(a, j)) for i, j in zip(houses, houses[1:])])
                    

In [27]:
groups = [
             ['dane',      'brit',   'swede',       'norwegian', 'german'   ],
             ['yellow',    'red',    'white',       'green',     'blue'     ],
             ['horse',     'cat',    'bird',        'fish',      'dog'      ],
             ['water',     'tea',    'milk',        'coffee',    'root_beer'],
             ['pall_mall', 'prince', 'blue_master', 'dunhill',   'blends'   ],
]

values = [value for group in groups for value in group]

In [28]:
cnf = []

# Each house gets exactly one value from each attribute group.
for house in houses:
    for group in groups:
        cnf += one_of([combine(value, house) for value in group])

# Each value gets assigned to exactly one house.
for value in values:
    cnf += one_of([combine(value, house) for house in houses])

# Encode the problem from the project document.
cnf += same('brit', 'red')
cnf += same('swede', 'dog')
cnf += same('dane', 'tea')
cnf += next_to('green', 'white')
cnf += same('green', 'coffee')
cnf += same('pall_mall', 'bird')
cnf += same('yellow', 'dunhill')
cnf += found_at('milk', 3)
cnf += found_at('norwegian', 1)
cnf += either_side('blends', 'cat')
cnf += either_side('horse', 'dunhill')
cnf += same('blue_master', 'root_beer')
cnf += same('german', 'prince')
cnf += either_side('norwegian', 'blue')
cnf += either_side('blends', 'water')

In [29]:
instance = Instance()

problem = cnf_to_string(cnf)
instance.parse_problem(problem)
instance.setup_watchlist()

assignment = [None] * len(instance.variables)

# Solve this problem with two_clause splitting heuristic.
sol, _ = solve(instance, assignment, two_clause_choice,
               unit_preference_methods.random_choice, False)

true_vars = [instance.variables[i].name for i in range(len(sol)) if sol[i]]

In [30]:
true_vars

['norwegian_1',
 'yellow_1',
 'cat_1',
 'water_1',
 'dunhill_1',
 'dane_2',
 'blue_2',
 'horse_2',
 'tea_2',
 'blends_2',
 'brit_3',
 'red_3',
 'bird_3',
 'milk_3',
 'pall_mall_3',
 'german_4',
 'green_4',
 'fish_4',
 'coffee_4',
 'prince_4',
 'swede_5',
 'white_5',
 'dog_5',
 'root_beer_5',
 'blue_master_5']